Variables used: r(request to website),we3(lxml from html),table(table of website),we3_table(pandas dataframe), we3(table name in database)

In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html

#### scraping data from html -1

In [2]:
r = requests.get("https://www.w3schools.com/html/html_tables.asp")

In [3]:
r

<Response [200]>

typing r.text and running it will give the text of source code which we saw by typing ^u

In [4]:
we3 = lxml.html.fromstring(r.text) #lxml is used to parse html content.

In [5]:
we3

<Element html at 0x7fe84f00d8b8>

Acquiring Table from "we3" 

In [6]:
table = we3.get_element_by_id("customers")

In [7]:
table

<Element table at 0x7fe84f029958>

In [8]:
table.getchildren() # we get the output as list which are table rows(tr)

[<Element tr at 0x7fe84e799458>,
 <Element tr at 0x7fe84e7993b8>,
 <Element tr at 0x7fe84e799368>,
 <Element tr at 0x7fe84e7994a8>,
 <Element tr at 0x7fe84e7994f8>,
 <Element tr at 0x7fe84e799548>,
 <Element tr at 0x7fe84e799598>]

In [9]:
len(table.getchildren()) # one is header and rest 6 are the table contents

7

In [10]:
table.getchildren()[0].getchildren() # this can be checked in the source code under the first set of <tr> tag

[<Element th at 0x7fe84e7997c8>,
 <Element th at 0x7fe84e799278>,
 <Element th at 0x7fe84e799318>]

In [11]:
table.getchildren()[0].text_content() # header of the table

'\r\n    Company\r\n    Contact\r\n    Country\r\n  '

In [12]:
print(table.getchildren()[6].getchildren()[0].text_content())# last row, 1st element
print(table.getchildren()[6].getchildren()[1].text_content())#last row, 2nd element
print(table.getchildren()[6].getchildren()[2].text_content())#last row, 3rd element

Magazzini Alimentari Riuniti
Giovanni Rovelli
Italy


#### parsing from lxml into pandas -2

In [13]:
data = {
    "Company"   :[],
    "Contact"   :[],
    "Country"   :[],    
}
for row in table.getchildren()[1:7]:
    data["Company"].append(row.getchildren()[0].text_content())
    data["Contact"].append(row.getchildren()[1].text_content())
    data["Country"].append(row.getchildren()[2].text_content())
    

In [14]:
we3_table = pd.DataFrame(data)
we3_table

,Company,Contact,Country
0,Alfreds Futterkiste,Maria Anders,Germany
1,Centro comercial Moctezuma,Francisco Chang,Mexico
2,Ernst Handel,Roland Mendel,Austria
3,Island Trading,Helen Bennett,UK
4,Laughing Bacchus Winecellars,Yoshi Tannamuri,Canada
5,Magazzini Alimentari Riuniti,Giovanni Rovelli,Italy


#### Inserting pandas data into sqlite -3

In [15]:
for i in we3_table.itertuples():
    print(i) 

Pandas(Index=0, Company='Alfreds Futterkiste', Contact='Maria Anders', Country='Germany')
Pandas(Index=1, Company='Centro comercial Moctezuma', Contact='Francisco Chang', Country='Mexico')
Pandas(Index=2, Company='Ernst Handel', Contact='Roland Mendel', Country='Austria')
Pandas(Index=3, Company='Island Trading', Contact='Helen Bennett', Country='UK')
Pandas(Index=4, Company='Laughing Bacchus Winecellars', Contact='Yoshi Tannamuri', Country='Canada')
Pandas(Index=5, Company='Magazzini Alimentari Riuniti', Contact='Giovanni Rovelli', Country='Italy')


In [16]:
for q in we3_table.itertuples(): #we dont want the Index as displayed above.
    print(q[1:]) 

('Alfreds Futterkiste', 'Maria Anders', 'Germany')
('Centro comercial Moctezuma', 'Francisco Chang', 'Mexico')
('Ernst Handel', 'Roland Mendel', 'Austria')
('Island Trading', 'Helen Bennett', 'UK')
('Laughing Bacchus Winecellars', 'Yoshi Tannamuri', 'Canada')
('Magazzini Alimentari Riuniti', 'Giovanni Rovelli', 'Italy')


In [17]:
conn = sqlite3.connect(":memory:") # database(stored in variable conn) is created temprorarily in memory
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE we3(Company, Contact, Country)
""")
for row in we3_table.itertuples():
    insert_table = """
    INSERT INTO we3(Company, Contact, Country) \
    VALUES (?,?,?)
"""
    cursor.execute(insert_table, row[1:])
#conn.close()
conn.commit()

#### Retrieving data from table we3 and display records using SELECT command-Data Query Language

In [18]:
retrieving = conn.execute("SELECT company, contact, country FROM we3")
for row in retrieving:
   print("Company = ", row[0])
   print("Contact = ", row[1])
   print("Country = ", row[2], "\n")

Company =  Alfreds Futterkiste
Contact =  Maria Anders
Country =  Germany 

Company =  Centro comercial Moctezuma
Contact =  Francisco Chang
Country =  Mexico 

Company =  Ernst Handel
Contact =  Roland Mendel
Country =  Austria 

Company =  Island Trading
Contact =  Helen Bennett
Country =  UK 

Company =  Laughing Bacchus Winecellars
Contact =  Yoshi Tannamuri
Country =  Canada 

Company =  Magazzini Alimentari Riuniti
Contact =  Giovanni Rovelli
Country =  Italy 



In [19]:
conn.close()